# SCADA Flow Rate Calculation Model

In [1]:
import pandas as pd
import os
import csv
import math

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Section 1

In this section, I read my Lift Station SCADA data and Lift Station Legend data to a pandas data frames and do some quick formatting. I'll begin by converting the date and time column to DateTime format, dropping unneeded columns, and dropping any NA values. Once this process is finished we are ready to begin section 2.

## Read SCADA data into Pandas Dataframe

In [2]:
df = pd.read_csv('LiftStation.csv')

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_3769/2210235391.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('LiftStation.csv')


In [3]:
legend = pd.read_csv('Lift Station Legend Cross Referance.csv')

In [4]:
legend = legend.drop('Pump #1 Run', axis=1)
legend = legend.drop('Pump #2 Run', axis=1)
legend = legend.drop('Pump #3 Run', axis=1)
legend = legend.drop('Table Tag #.4', axis=1)
legend = legend.drop('Pump #4 Run', axis=1)
legend = legend.drop('Unnamed: 11', axis=1)

## Convert the DateAndTime column to Datetime Format

In [5]:
df['PumpStart'] = pd.to_datetime(df['PumpStart'])
df['PumpStop'] = pd.to_datetime(df['PumpStop'])

## Format the dataframe

In [6]:
df = df.dropna(subset=['PumpStart', 'PumpStop'])

In [7]:
df['PumpTagIndex'] = df['PumpTagIndex'].astype(int)

In [8]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = df.duplicated(subset =['PumpTagIndex', 'PumpStart'], keep = 'first')

# drop the duplicate rows using the boolean mask

#now let's drop the duplicate rows
new_df = df[~mask]
# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# print the length of the new dataframe
#print(len(new_df))

In [9]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = new_df.duplicated(subset=['PumpTagIndex', 'PumpStop'], keep='first')

# drop the duplicate rows using the boolean mask
new_df = new_df[~mask]

# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# print the length of the new dataframe
#print(len(new_df))

# Section 2

In this section, I will create a new data frame called calc_df, which will hold all of our calculated data going forward. 

I'll begin by assigning the original df to my calculated df, and reformatting the columns (names and order). 

Next I will remove any erroneous rows containing invalid pump numbers.

Then I'll use the legend dataframe I read into the program earlier to add the correct lift station name to each row in calc_df.

Finally, I will convert both on and off timestamps into Unix time and calculate the difference in time between the pump turning on and off. 

Note: Remove the # in front of any of the calc_df lines to see a preview of the data frame.

## Create new data frame

In [10]:
calc_df = new_df
calc_df = calc_df.rename(columns={'PumpStart': 'PumpStartTime'})
calc_df = calc_df.rename(columns={'PumpStop': 'PumpStopTime'})
calc_df = calc_df.rename(columns={'WellStart': 'StartWellLevel'})
calc_df = calc_df.rename(columns={'WellEnd': 'StopWellLevel'})

In [11]:
calc_df['LiftStation'] = None
calc_df['StartUnix'] = 0.0
calc_df['StopUnix'] = 0.0
calc_df['Time_diff'] = 0.0

In [12]:
calc_df = calc_df[['LiftStation','PumpTagIndex', 'PumpStartTime', 'StartWellLevel','PumpStopTime', 'StopWellLevel', 'StartUnix', 'StopUnix', 'Time_diff']]

In [13]:
#calc_df

## Delete any rows with incorrect pump numbers

In [14]:
#create a list of valid pump numbers

valid_pumps = []

for index, row in legend.iterrows():
    valid_pumps.append(row['Table Tag #'])
    valid_pumps.append(row['Table Tag #.1'])
    valid_pumps.append(row['Table Tag #.2'])
    valid_pumps.append(row['Table Tag #.3'])
    
valid_pumps = list(filter(lambda x: not math.isnan(x), valid_pumps))
        
#len(valid_pumps)

In [15]:
calc_df = calc_df[calc_df['PumpTagIndex'].isin(valid_pumps)]
calc_df = calc_df.reset_index(drop=True)
calc_df['PumpTagIndex'] = calc_df['PumpTagIndex'].astype(int)

## Add the correct Lift Station to each row in calc_df

Here I am going to perfom 4 joins between the legend dataframe and my calc_df. Each join will merge the legend and calc data frames on one of the 4 table tag #'s.

Table Tag 0 Join

In [16]:
# Left join calc_df to legend df where Pump Tag Index = Table Tag
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 1 Join

In [17]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.1', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 2 Join

In [18]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.2', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 3 Join

In [19]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.3', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

## Now let's get Unix time for each timestamp

In [20]:
calc_df['StartUnix'] = calc_df['PumpStartTime'].astype(int)/ 10**9
calc_df['StopUnix'] = calc_df['PumpStopTime'].astype(int) / 10**9

In [21]:
#calc_df.head(20)

## Calculating the time difference between Pumps turning on and off

In [22]:
calc_df['Time_diff'] = (calc_df['StopUnix'] - calc_df['StartUnix']) / 60

# Section 3

In this section, I will add elevation difference, volume per foot, observed pumping flow rate, the average flow rate per pump, and average observed pumping flow rate (GPM).

Elevation Difference - This value can be changed for the desired lift station. Please note that each lift station will have a different elevation level. To change the elevation for a different lift station, simply change the 2.2 in the first cell below.

Observed pumping flow rate - To calculate this value we first need to input the wet wells diameter, again this value will differ per each lift station. Simply change the diameter in the diam_well = (5) statement to the desired value. From there the vol_per_foot calculation will update and be added to the calc_df. Now we have all the information in the calc_df to calculate the observed pumping flow rate. I'll take the elevation difference value times the volume per foot value, and divide the result by our time difference. The resulting value will be added to the calc_df in the observed pumping flow rate column.

Average flow rate per pump - Average flow rate per pump is calculated by adding the values in the observed pumping flow rate column for each pump and then dividing that sum by the count of values for that pump. So if I iterate over 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an observed pumping flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the number of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average observed pumping flow rate (GPM) - to calculate this value I simply add all the average flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example, if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, I'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average observed pumping flow rate in GPM of 107.5 in row 4.



## Add in Elevation Difference

In [23]:
calc_df['Elevation_dif'] = 0.0

In [24]:
legend['Elevation_diff'] = 2.2

In [25]:
legend.loc[0, 'Elevation_diff'] = 3.3

In [26]:
#merge legend elevation_diff to calc_df

merged_df = calc_df.merge(legend, left_on = 'LiftStation', right_on = 'Lift Station', how = 'left')

mask = merged_df['Elevation_diff'].notnull()

calc_df.loc[mask, 'Elevation_dif'] = merged_df.loc[mask, 'Elevation_diff']

## Calculate Observed Pumping Flow Rate

### Calculate Volume Per Foot for each well

In [27]:
#initialize new column
calc_df['wet_well_diam'] = 0.0

# left join calc_df and legend df on lift station
merged_df = calc_df.merge(legend, left_on = 'LiftStation', right_on = 'Lift Station', how = 'left')

#create a boolean mask to find all rows where we have a wet well diameter
mask = merged_df['Wet Well Diam'].notnull()

#add the wet well diameter value from the legend dataframe to the calc_df where mask is true
calc_df.loc[mask, 'wet_well_diam'] = merged_df.loc[mask, 'Wet Well Diam']

In [28]:
#Calculate volume per foot (check 4*7.48)
calc_df['volume_per_foot'] = round(((calc_df['wet_well_diam']*calc_df['wet_well_diam']) * 3.14)/4*7.48)

### Calculate Observed Pumping Flow Rate

In [29]:
calc_df['observed_pumping_flow_rate'] = int()

In [30]:
for index, row in calc_df.iterrows():
    result = round((row['Elevation_dif'] * row['volume_per_foot'])/row['Time_diff'])
    calc_df.at[index,'observed_pumping_flow_rate'] = result

In [31]:
#calc_df

## Calculate Average Flow Rate Per Pump

In [32]:
calc_df['avg_flow_rate_pump'] = int()

### Create two dictionaries
### 1. Dictionary containing a running total of Observed Pumping Flow Rate for each pump
### 2. Dictionary containing a running row count of Observed Pumping Flow Rate for each pump

In [33]:
sum_dict ={}
count_dict ={}

for item in valid_pumps:
    result = 0
    sum_dict[item] = result
    count_dict[item] = result

### Calculate Average Flow Rate Per Pump

In [34]:
for index, row in calc_df.iterrows():
    tag = row["PumpTagIndex"]
    sum_dict[tag] += row['observed_pumping_flow_rate']
    count_dict[tag] += 1
    calc_df.at[index, 'avg_flow_rate_pump'] = sum_dict[tag] / count_dict[tag]    

## Finally Let's calculate our Average Observed Pumping Flow Rate (GPM)

In [35]:
calc_df['average_observed_pumping_flow_rate'] = int()

### Create a dictionary for sum and count for each Lift Station

In [36]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result

### Calculate Average Observed Pumping Flow Rate (GPM)

In [37]:
for index, row in calc_df.iterrows():
    tag = row["LiftStation"]
    sum_dict[tag] += row['observed_pumping_flow_rate']
    count_dict[tag] += 1
    calc_df.at[index, 'average_observed_pumping_flow_rate'] = sum_dict[tag] / count_dict[tag]  

In [38]:
#filtered_df = calc_df[calc_df['LiftStation'] == 'Waldo (22)']
#filtered_df.head(60)

# Section 4

In this section, I will calculate the time to fill, inflow, and average inflow. These values need to be accounted for when calculating flow rate, as even when the pumps are turned on within a wet well and the level is dropping, water is still flowing through the sewer pipes into the well.

Time to fill - To calculate the time to fill for each row in calc_df, I take the stop Unix time of the first row within calc_df (when the pump turned off) and subtract this value from the start Unix time of the next row (when the pump turned back on). The difference between the pump turning off and the next pump turning on is my time to fill the measure.

Inflow - To calculate my inflow value for each row in calc_df, I take the row's volume per foot value and multiply it by the row's elevation difference value. I then divide that result by the row's time to fill value we just calculated.

Average Inflow - To calculate my average inflow for each row in calc_df, I simply add all the values for inflow in the rows I have iterated over and divide by the count of those rows. So if I have iterated over 4 rows so far in calc_df and the inflow values are (60,40,20,60) I sum these values (180) and divide by four to get my average inflow value in row 4, 45.


## Calculate time to fill

In [39]:
calc_df['Time_to_Fill'] = float()

### Create two dictionaries
### 1. Dictionary containing the Start Unix time for each Lift Station
### 2. Dictionary containing the Stop Unix time for each Lift Station

In [40]:
# Sort the dataframe by LiftStation and PumpStartTime
calc_df = calc_df.sort_values(by=['LiftStation', 'PumpStartTime'])
calc_df = calc_df.reset_index(drop=True)

In [41]:
start_list = []
stop_list = []
lift_station = []
index = 0

for index, row in calc_df.iterrows():
    if len(stop_list) == 0:
        stop_list.append(row['StopUnix'])
        lift_station.append(row['LiftStation'])
        index += 1

    if len(stop_list) == 1 and lift_station[-1] == row['LiftStation']:
        start_list.append(row['StartUnix'])
        result = (start_list[0] - stop_list[0]) * 60
        calc_df.at[index, 'Time_to_Fill'] = result
        del stop_list[0]
        del start_list[0]
        stop_list.append(row['StopUnix'])
        index += 1
        
    else:
        del stop_list[0]
        stop_list.append(row['StopUnix'])
        lift_station.append(row['LiftStation'])
        index += 1
        


In [42]:
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] < 0:
        result = 0
        calc_df.at[index, 'Time_to_Fill'] = result

In [43]:
negative_values = calc_df.loc[calc_df['Time_to_Fill'] < 0, 'Time_to_Fill']
negative_values

Series([], Name: Time_to_Fill, dtype: float64)

## Calculate Inflow and Average Inflow

In [45]:
calc_df['Inflow'] = float()
calc_df['Avg_Inflow'] = float()

In [46]:
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] != 0:
        inflow = row['volume_per_foot'] * row['Elevation_dif'] / row['Time_to_Fill']
        calc_df.at[index, 'Inflow'] = inflow
    else:
        pass

In [47]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result
    
for index, row in calc_df.iterrows():
    tag = row['LiftStation']
    sum_dict[tag] += row['Inflow']
    count_dict[tag] += 1
    calc_df.at[index, 'Avg_Inflow'] = sum_dict[tag] / count_dict[tag]


In [52]:
#calc_df.head(10)

# Section 5

In this section, I will calculate the actual pump flow rate in GPM, the average flow rate per pump in GPM, and the average pump flow rate in GPM.

Actual pump flow rate - To calculate the actual pump flow rate for each row in calc_df, I simply take the row's observed pumping flow rate value we calculated earlier, and add the row's inflow value.

Average flow rate per pump - To calculate the average flow rate per pump I add the values in the actual pump flow rate column for each pump, and then divide that sum by the count of values for that pump. So if I iterate over 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an actual pump flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the number of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average pump flow rate - to calculate this value I simply add all the actual pump flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example, if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, I'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average pump flow rate in GPM of 107.5.

## Calculate Actual Pump Flow Rate (GPM)

In [49]:
calc_df['Actual_Pump_Flow_Rate'] = float()

In [50]:
for index, row in calc_df.iterrows():
    result = round(row['observed_pumping_flow_rate'] + row['Inflow'])
    calc_df.at[index, 'Actual_Pump_Flow_Rate'] = result

## Finally Let's calculate our Average Flow Rate Per Pump and the Average Pump Flow Rate in (GPM)

In [58]:
calc_df['Average_Pump_1_Flow_Rate'] = int()
calc_df['Average_Pump_2_Flow_Rate'] = int()
calc_df['Average_Pump_3_Flow_Rate'] = int()
calc_df['Average_Pump_4_Flow_Rate'] = int()
calc_df['Average_Pump_Flow_Rate(all)'] = int()

In [55]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    if row['Pump_num'] == 974:
        cum_sum += row['Actual_Pump_Flow_Rate']
        count += 1
        calc_df.at[index,'Average_Pump_1_Flow_Rate'] = cum_sum / count

In [56]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    if row['Pump_num'] == 975:
        cum_sum += row['Actual_Pump_Flow_Rate']
        count += 1
        calc_df.at[index,'Average_Pump_2_Flow_Rate'] = cum_sum / count

In [57]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    cum_sum += row['Actual_Pump_Flow_Rate']
    count += 1
    calc_df.at[index,'Average_Pump_Flow_Rate(1&2)'] = cum_sum / count

In [44]:
#calc_df

## Export the data to CSV

In [39]:
calc_df.to_csv("Model Data for Kelly Lift Station.csv")